In [1]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2
resnet_weights_path = './resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(num_classes, activation='softmax'))
    
# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

Instructions for updating:
Colocations handled automatically by placer.


In [2]:
my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [3]:
from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

image_size = 224
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = data_generator.flow_from_directory(
        './train',
        target_size=(image_size, image_size),
        batch_size=24,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        './val',
        target_size=(image_size, image_size),
        class_mode='categorical')

my_new_model.fit_generator(
        train_generator,
        steps_per_epoch=3,
        validation_data=validation_generator,
        validation_steps=1)

Found 88 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.
4/4 [==============================] - 10s 2s/step - loss: 0.7871 - acc: 0.5682 - val_loss: 1.3753 - val_acc: 0.5000


In [4]:
my_new_model.save("model.h5")

In [5]:
from tensorflow.python.keras.models import load_model
import cv2
import argparse
import os
import numpy as np

In [6]:
image_size = 224

In [7]:
testfileName = "test-pos.jpg"

In [8]:
classifier = load_model('model.h5')
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', 
metrics = ['accuracy'])

In [9]:
# make prediction if image specified exists
if(os.path.isfile(testfileName)):
    img = cv2.imread(testfileName)
    img = cv2.resize(img, (image_size, image_size))
    img = img.astype("float") /  255.0
    img = np.reshape(img, [1, image_size, image_size, 3])

    pred = classifier.predict(img)
    
    predictedClass = "UNRECOGNIZABLE"
    print ("pred", pred)
    
    if(pred[0][0] <= 0.5):
        predictedClass = "No hat"
        print("The predicted class is: ", predictedClass)
        print("The model's predicted score is: ", pred[0][1])
    else:
        predictedClass = "Hard hat"
        print("The predicted class is: ", predictedClass)
        print("The model's predicted score is: ", pred[0][0])
    
else:
    print("The directory " + fileName + " could not be located.")

pred [[0.7511718  0.24882819]]
The predicted class is:  Hard hat
The model's predicted score is:  0.7511718
